In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers, Model
import random
from sklearn.metrics import accuracy_score
from numpy.linalg import norm
import time

In [3]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),      # lật ngang ngẫu nhiên
    layers.RandomRotation(0.1),           # quay ±10%
    layers.RandomZoom(0.1),               # zoom ngẫu nhiên trong khoảng ±10%
    layers.RandomTranslation(0.1, 0.1),    # dịch chuyển ±10% chiều cao & rộng
    layers.RandomContrast(0.1),           # thay đổi độ tương phản ±10%
])

In [4]:
def build_embedding_model(input_shape=(160,160,3), embedding_dim=128):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)

    embeddings = layers.Dense(embedding_dim, activation=None, name='embeddings')(x)
    embeddings = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(embeddings)

    return Model(inputs, embeddings, name='embedding_model')

In [5]:
def build_classification_model(embedding_model, num_classes):
    x = embedding_model.output
    outputs = layers.Dense(num_classes, activation='softmax', name='softmax')(x)
    return Model(embedding_model.input, outputs, name='classification_model')


In [6]:
train_dir = "/Users/hqpl/Downloads/archive/train" # Đường dẫn đến thư mục
val_dir = "/Users/hqpl/Downloads/archive/val" # Đường dẫn đến thư mục


In [7]:
# 1) Khởi tạo embedding model
emb_model = build_embedding_model(input_shape=(160,160,3), embedding_dim=128)
pre_weights = emb_model.get_weights()

In [8]:
# 2) Tạo classification model
num_classes = len(os.listdir(train_dir))  # số người
cls_model = build_classification_model(emb_model, num_classes)

In [9]:
# 3) Compile với Sparse Categorical Crossentropy
cls_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [9]:
# 4) Load data bằng image_dataset_from_directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=(160,160), batch_size=32, label_mode='int', shuffle=True, seed=123
).map(
    lambda x, y: (data_augmentation(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,   image_size=(160,160), batch_size=32, label_mode='int', shuffle=False)

Found 176396 files belonging to 480 classes.
Found 21294 files belonging to 60 classes.


In [ ]:
# 5) Huấn luyện
cls_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("best_cls.h5", save_best_only=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=3)
    ]
)

Epoch 1/5
5513/5513 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.0186 - loss: 5.6620

5513/5513 ━━━━━━━━━━━━━━━━━━━━ 1080s 196ms/step - accuracy: 0.0186 - loss: 5.6620 - val_accuracy: 4.6962e-05 - val_loss: 6.4288 - learning_rate: 1.0000e-04
Epoch 2/5
5247/5513 ━━━━━━━━━━━━━━━━━━━━ 50s 192ms/step - accuracy: 0.0228 - loss: 5.4627

In [10]:
post_weights = emb_model.get_weights()
changed = False
for idx, (w_pre, w_post) in enumerate(zip(pre_weights, post_weights)):
    if not np.allclose(w_pre, w_post):
        print(f"Layer {idx} has changed the weight (diff normal = {np.linalg.norm(w_pre - w_post):.6f})")
        changed = True

if not changed:
    print("No weight changes — may be training not running or weights not updated.")
else:
    print("✅ Emb_model weights has been updated after training.")

No weight changes — may be training not running or weights not updated.


In [11]:
# Load weights tốt nhất vào classification model
cls_model.load_weights("best_cls.h5")
print(cls_model.summary())

Model: "classification_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 160, 160, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 80, 80, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 40, 40, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 40, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings (Dense)              │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Dense)                 │ (None, 480)            │        61,920 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209,248 (817.38 KB)

 Trainable params: 208,928 (816.12 KB)

 Non-trainable params: 320 (1.25 KB)

None


In [12]:
emb_model.save("face_embedding_model_64.keras")
emb_model.save("face_embedding_model_64.h5")
print(emb_model.summary())

Model: "embedding_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 160, 160, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 80, 80, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 40, 40, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 40, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embeddings (Dense)              │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 128)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147,328 (575.50 KB)

 Trainable params: 147,008 (574.25 KB)

 Non-trainable params: 320 (1.25 KB)

None


In [ ]:
def preprocess_image(img_path, target_size=(160,160)):
    # Đọc ảnh BGR và chuyển sang RGB
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    # Resize
    img = cv2.resize(img, target_size)
    # Chuyển sang float và normalize về [-1,1]
    img = img.astype(np.float32)
    img = (img / 127.5) - 1.0
    # Thêm batch dimension
    img = np.expand_dims(img, axis=0)  # shape (1, H, W, C)
    return img

In [21]:
# Đường dẫn tới ảnh muốn test
img_path = "/Users/hqpl/Downloads/archive/val/n000001/0001_01.jpg"
x = preprocess_image(img_path)
# predict trả về array shape (1, embedding_dim)
emb_vector = emb_model.predict(x)
# bỏ batch dimension nếu cần
emb_vector = emb_vector[0]
print("Embedding vector shape:", emb_vector.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Embedding vector shape: (128,)


In [25]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

# Suppose there are two test photos
emb1 = emb_model.predict(preprocess_image("/Users/hqpl/Downloads/archive/val/n000001/0001_01.jpg"))[0]
emb2 = emb_model.predict(preprocess_image("/Users/hqpl/Downloads/archive/val/n000001/0005_01.jpg"))[0]
sim = cosine_similarity(emb1, emb2)
print(f"Cosine similarity = {sim:.3f}")
# Thresholding: for example sim > 0.5 is the same person (subject to inspection)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Cosine similarity = 0.346


In [16]:

# Suppose the emb_model has been load or initialized 
# emb_model = tf.keras.models.load_model ("face_embedding_model")
 
# Create a fake batch to measure speed (vd. batch_size=32)
batch_size = 32
dummy_batch = np.random.randn(batch_size, 160, 160, 3).astype(np.float32)

# 1) Warm‑up
_ = emb_model.predict(dummy_batch)

# 2) Đo inference
n_runs = 100
start = time.time()
for _ in range(n_runs):
    _ = emb_model.predict(dummy_batch)
end = time.time()

avg_time = (end - start) / n_runs
print(f"Average inference time per batch of {batch_size}: {avg_time:.4f} seconds")
print(f"  => Per image: {avg_time / batch_size:.4f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

In [25]:
# 2) Chuẩn bị list ảnh
img_dir = "/Users/hqpl/Downloads/archive/val/n000001"
img_paths = [
    os.path.join(img_dir, fn)
    for fn in os.listdir(img_dir)
    if fn.lower().endswith(('.jpg','.png','.jpeg'))
]

# 3) Hàm tiền xử lý ảnh
def preprocess_image(img_path, target_size=(160,160)):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size).astype(np.float32)
    img = (img / 127.5) - 1.0
    # thêm batch dim
    return np.expand_dims(img, axis=0)  # shape (1, H, W, C)

# 4) Warm‑up (build graph, load weights vào cache)
sample = preprocess_image(img_paths[0])
_ = emb_model(sample, training=False)

# 5) (Tuỳ chọn) Wrap inference vào tf.function để chạy nhanh hơn
infer_fn = tf.function(lambda x: emb_model(x, training=False),
                       input_signature=[tf.TensorSpec([1,160,160,3], tf.float32)])
_ = infer_fn(sample)  # warm‑up graph

# 6) Đo thời gian per‑image
total_time = 0.0
for p in img_paths:
    img = preprocess_image(p)
    start = time.perf_counter()
    _ = infer_fn(img)
    end = time.perf_counter()
    total_time += (end - start)

n_images = len(img_paths)
print(f"Processed {n_images} images in {total_time:.4f} seconds")
print(f"Average per image: {total_time / n_images:.4f} seconds")

Processed 424 images in 1.2917 seconds
Average per image: 0.0030 seconds
